# 1st Assignment

Marina Samprovalaki f3322310  </n>

Petros Chanas f3322314


In the 1st Assignment we had to create some n-gram language models in order to implement a context-aware spelling corrector

We assume that each sentence starts with the
pseudo-token *start* (or two pseudo-tokens *start1*, *start2* for the trigram model) and ends with the pseudo-token *end*.


# Libraries Installation

In [1]:
!pip install -U nltk

In [2]:
pip install tabulate

In [3]:
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer

import random
import math
from tabulate import tabulate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

# Selecting a corpus

A corpus is a collection of text that serves as a valuable resource for language analysis, training models, and conducting research. As per the assignment's description, we recommend choosing a corpus from the [link](http://www.nltk.org/). We selected the Alice's Adventures in Wonderland, a novel by Lewis Carroll. We found this corpus in the Gutenberg Corpus​​ NLTK.

In [5]:
nltk.download("gutenberg")
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [6]:
# Access and print the text of "Alice's Adventures in Wonderland"
alice_text = gutenberg.raw("carroll-alice.txt")
print(alice_text[:500])  # Print the first 500 characters as a sample


[Alice's Adventures in Wonderland by Lewis Carroll 1865]

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversation?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy an


## Lowercases

In [7]:
# Convert to lowercase and split by newlines
alice_text = alice_text.lower().split('\n')
alice_text

["[alice's adventures in wonderland by lewis carroll 1865]",
 '',
 'chapter i. down the rabbit-hole',
 '',
 'alice was beginning to get very tired of sitting by her sister on the',
 'bank, and of having nothing to do: once or twice she had peeped into the',
 'book her sister was reading, but it had no pictures or conversations in',
 "it, 'and what is the use of a book,' thought alice 'without pictures or",
 "conversation?'",
 '',
 'so she was considering in her own mind (as well as she could, for the',
 'hot day made her feel very sleepy and stupid), whether the pleasure',
 'of making a daisy-chain would be worth the trouble of getting up and',
 'picking the daisies, when suddenly a white rabbit with pink eyes ran',
 'close by her.',
 '',
 'there was nothing so very remarkable in that; nor did alice think it so',
 "very much out of the way to hear the rabbit say to itself, 'oh dear!",
 "oh dear! i shall be late!' (when she thought it over afterwards, it",
 'occurred to her that she oug

# Dividing the dataset

To facilitate the development and evaluation of n-gram models, it is essential to divide our corpus into three distinct subsets. The train set, used for training, the valid set for validation, and the test set for conducting tests, have been created with shuffling to prevent biases.


In [8]:
import random

# Define the ratios for splitting
train_ratio = 0.7  # 70% for training
valid_ratio = 0.15  # 15% for validation
test_ratio = 0.15  # 15% for testing


# Shuffle the alice_text to ensure randomness
random.shuffle(alice_text)

# Calculate the split points based on the ratios
total_alice_text = len(alice_text)
train_split = int(total_alice_text * train_ratio)
valid_split = train_split + int(total_alice_text * valid_ratio)

# Split the alice_text into subsets
train_set = alice_text[:train_split]
valid_set = alice_text[train_split:valid_split]
test_set = alice_text[valid_split:]

# Join the alice_text back into text if needed
train_text = '\n'.join(train_set)
valid_text = '\n'.join(valid_set)
test_text = '\n'.join(test_set)


In [9]:
train_text

'\'if you knew time as well as i do,\' said the hatter, \'you wouldn\'t talk\n\nsilent.\n\n\ndrunk quite so much!\'\nof things--i can\'t remember half of them--and it belongs to a farmer,\nbefore she made her next remark. \'then the eleventh day must have been a\n\n\nlittle thing grunted in reply (it had left off sneezing by this time).\nslate with one finger, as he found it made no mark; but he now hastily\n\nover all the children she knew that were of the same age as herself, to\nyou must cross-examine the next witness. it quite makes my forehead\nand was just going to leave the room, when her eye fell upon a little\nof cherry-tart, custard, pine-apple, roast turkey, toffee, and hot\nshriek of the gryphon, and all the other queer noises, would change (she\nnobody in particular. \'she\'d soon fetch it back!\'\n\nthe executioner, the king, and the queen, who were all talking at once,\n but the snail replied "too far, too far!" and gave a look askance--\nbegan fancying the sort of thing

In [10]:
valid_text

'\'oh, i beg your pardon!\' she exclaimed in a tone of great dismay, and\nbut at any rate he might answer questions.--how am i to get in?\' she\n\n\nit sounded an excellent plan, no doubt, and very neatly and simply\n\n\n\n        or judge,\n\n\nhis fan and gloves--that is, if i can find them.\' as she said this, she\nsays it kills all the rats and--oh dear!\' cried alice in a sorrowful\nconclusion that it was a very difficult game indeed.\n\n\nthe teapot.\n\'hold your tongue!\' said the queen, turning purple.\n\n\'please your majesty,\' said the knave, \'i didn\'t write it, and they\n\'that\'s the reason they\'re called lessons,\' the gryphon remarked:\n\n\'but about his toes?\' the mock turtle persisted. \'how could he turn them\nback to finish his story.\ndoesn\'t like changing so often, you know.\'\n\n\n\n\'in that case,\' said the dodo solemnly, rising to its feet, \'i move\n\n\n\'once upon a time there were three little sisters,\' the dormouse began\n\n\'shall we try another figu

In [11]:
test_text

'listen, the whole place around her became alive the strange creatures of\n\'wouldn\'t it really?\' said alice in a tone of great surprise.\nalternately without a moment\'s pause. the only things in the kitchen\nuncomfortably sharp chin. however, she did not like to be rude, so she\nshe felt a little nervous about this; \'for it might end, you know,\' said\n\n\nof tears, \'i do wish they would put their heads down! i am so very tired\nproposal.\nwas a table, with a large dish of tarts upon it: they looked so good,\nfrightened mouse splashed his way through the neighbouring pool--she\n\'that\'s very important,\' the king said, turning to the jury. they were\n\'you\'re wrong about the crumbs,\' said the mock turtle: \'crumbs would all\n\'ten hours the first day,\' said the mock turtle: \'nine the next, and so\n     like a tea-tray in the sky.\nadded the gryphon; and then they both sat silent and looked at poor\n\'--yes, that\'s about the right distance--but then i wonder what latitude\n\

# Train Subset Preprocessing

In [12]:
train_text

'\'if you knew time as well as i do,\' said the hatter, \'you wouldn\'t talk\n\nsilent.\n\n\ndrunk quite so much!\'\nof things--i can\'t remember half of them--and it belongs to a farmer,\nbefore she made her next remark. \'then the eleventh day must have been a\n\n\nlittle thing grunted in reply (it had left off sneezing by this time).\nslate with one finger, as he found it made no mark; but he now hastily\n\nover all the children she knew that were of the same age as herself, to\nyou must cross-examine the next witness. it quite makes my forehead\nand was just going to leave the room, when her eye fell upon a little\nof cherry-tart, custard, pine-apple, roast turkey, toffee, and hot\nshriek of the gryphon, and all the other queer noises, would change (she\nnobody in particular. \'she\'d soon fetch it back!\'\n\nthe executioner, the king, and the queen, who were all talking at once,\n but the snail replied "too far, too far!" and gave a look askance--\nbegan fancying the sort of thing

## Vocabulary

In [13]:
import nltk
from collections import Counter

# Tokenize the text
words = nltk.word_tokenize(train_text)

# Count word frequencies
word_frequencies = Counter(words)

# Sort word frequencies by count in descending order
sorted_word_frequencies = dict(sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True))

# Print sorted word frequencies
for word, frequency in sorted_word_frequencies.items():
    print(f"{word}: {frequency}")


,: 1656
the: 1118
': 877
.: 682
and: 570
to: 495
a: 423
she: 389
it: 379
of: 360
i: 356
said: 307
!: 300
alice: 267
in: 265
was: 255
you: 229
that: 193
--: 179
as: 177
her: 164
:: 162
at: 144
;: 137
n't: 136
?: 135
's: 132
had: 126
all: 125
on: 121
with: 114
be: 107
for: 101
but: 100
very: 98
not: 96
this: 93
they: 90
so: 89
he: 89
do: 87
little: 85
out: 82
is: 79
down: 74
up: 73
one: 68
them: 67
what: 66
there: 65
would: 64
about: 64
know: 63
his: 62
have: 61
could: 61
again: 60
like: 60
when: 57
were: 56
herself: 55
went: 55
did: 55
'': 54
if: 53
thought: 53
no: 52
queen: 52
then: 51
see: 50
time: 48
king: 48
or: 47
began: 46
into: 46
me: 45
by: 42
how: 42
say: 42
quite: 41
its: 41
your: 41
'll: 41
much: 40
off: 40
): 40
'and: 40
'm: 40
'it: 40
hatter: 39
(: 39
my: 39
who: 39
more: 39
'you: 38
here: 38
way: 37
voice: 37
some: 37
turtle: 37
go: 37
got: 37
now: 36
mock: 36
just: 34
are: 34
*: 34
must: 33
gryphon: 33
rabbit: 33
him: 33
an: 33
head: 33
first: 32
two: 32
came: 32
before: 

In [14]:
# Filter words with frequency > 10
filtered_word_frequencies = {word: frequency for word, frequency in word_frequencies.items() if frequency > 10}

# Sort word frequencies by count in descending order
sorted_word_frequencies = dict(sorted(filtered_word_frequencies.items(), key=lambda item: item[1], reverse=True))


In [15]:
vocab = filtered_word_frequencies
vocab

{"'if": 13,
 'you': 229,
 'time': 48,
 'as': 177,
 'well': 25,
 'i': 356,
 'do': 87,
 ',': 1656,
 "'": 877,
 'said': 307,
 'the': 1118,
 'hatter': 39,
 "'you": 38,
 'would': 64,
 "n't": 136,
 '.': 682,
 'quite': 41,
 'so': 89,
 'much': 40,
 '!': 300,
 'of': 360,
 'things': 21,
 '--': 179,
 'ca': 17,
 'half': 16,
 'them': 67,
 'and': 570,
 'it': 379,
 'to': 495,
 'a': 423,
 'before': 31,
 'she': 389,
 'made': 25,
 'her': 164,
 'next': 16,
 'day': 17,
 'must': 33,
 'have': 61,
 'been': 30,
 'little': 85,
 'thing': 29,
 'in': 265,
 '(': 39,
 'had': 126,
 'off': 40,
 'by': 42,
 'this': 93,
 ')': 40,
 'with': 114,
 'one': 68,
 'he': 89,
 'found': 25,
 'no': 52,
 ';': 137,
 'but': 100,
 'now': 36,
 'hastily': 12,
 'over': 29,
 'all': 125,
 'that': 193,
 'were': 56,
 'same': 18,
 'herself': 55,
 'my': 39,
 'was': 255,
 'just': 34,
 'going': 19,
 'when': 57,
 'upon': 21,
 'gryphon': 33,
 'other': 27,
 "'d": 17,
 'soon': 17,
 'back': 25,
 'king': 48,
 'queen': 52,
 'who': 39,
 'talking': 12,
 '

In [16]:
len(vocab)

271

## OOV

In [17]:
filtered_words = [word if word_frequencies[word] > 10 else "UNK" for word in words]
filtered_words

["'if",
 'you',
 'UNK',
 'time',
 'as',
 'well',
 'as',
 'i',
 'do',
 ',',
 "'",
 'said',
 'the',
 'hatter',
 ',',
 "'you",
 'would',
 "n't",
 'UNK',
 'UNK',
 '.',
 'UNK',
 'quite',
 'so',
 'much',
 '!',
 "'",
 'of',
 'things',
 '--',
 'i',
 'ca',
 "n't",
 'UNK',
 'half',
 'of',
 'them',
 '--',
 'and',
 'it',
 'UNK',
 'to',
 'a',
 'UNK',
 ',',
 'before',
 'she',
 'made',
 'her',
 'next',
 'UNK',
 '.',
 'UNK',
 'the',
 'UNK',
 'day',
 'must',
 'have',
 'been',
 'a',
 'little',
 'thing',
 'UNK',
 'in',
 'UNK',
 '(',
 'it',
 'had',
 'UNK',
 'off',
 'UNK',
 'by',
 'this',
 'time',
 ')',
 '.',
 'UNK',
 'with',
 'one',
 'UNK',
 ',',
 'as',
 'he',
 'found',
 'it',
 'made',
 'no',
 'UNK',
 ';',
 'but',
 'he',
 'now',
 'hastily',
 'over',
 'all',
 'the',
 'UNK',
 'she',
 'UNK',
 'that',
 'were',
 'of',
 'the',
 'same',
 'UNK',
 'as',
 'herself',
 ',',
 'to',
 'you',
 'must',
 'UNK',
 'the',
 'next',
 'UNK',
 '.',
 'it',
 'quite',
 'UNK',
 'my',
 'UNK',
 'and',
 'was',
 'just',
 'going',
 'to',


In [18]:
# Count OOV words
oov_count = filtered_words.count("UNK")

print("Number of OOV words:", oov_count)

Number of OOV words: 4791


In [19]:
train_text = " ".join([word if word_frequencies[word] > 10 else "UNK" for word in words])
train_text

"'if you UNK time as well as i do , ' said the hatter , 'you would n't UNK UNK . UNK quite so much ! ' of things -- i ca n't UNK half of them -- and it UNK to a UNK , before she made her next UNK . UNK the UNK day must have been a little thing UNK in UNK ( it had UNK off UNK by this time ) . UNK with one UNK , as he found it made no UNK ; but he now hastily over all the UNK she UNK that were of the same UNK as herself , to you must UNK the next UNK . it quite UNK my UNK and was just going to UNK the UNK , when her UNK UNK upon a little of UNK , UNK , UNK , UNK UNK , UNK , and UNK UNK of the gryphon , and all the other UNK UNK , would UNK ( she UNK in UNK . UNK 'd soon UNK it back ! ' the UNK , the king , and the queen , who were all talking at once , but the UNK replied `` too UNK , too UNK ! '' and gave a look UNK -- began UNK the sort of thing that would UNK : ' '' UNK alice ! come all her UNK . UNK one as long as you UNK . ' the gryphon UNK up and UNK its eyes : then it UNK the quee

In [20]:
def replace_oov_in_text(text):
    # Tokenize the input text into words
    tokens = word_tokenize(text)

    # Replace OOV words with "UNK" and join them back into plain text
    plain_text = ' '.join(["UNK" if token not in vocab else token for token in tokens])

    return plain_text

In [21]:
valid_text = replace_oov_in_text(valid_text)
valid_text

"'oh , i UNK your UNK ! ' she UNK in a tone of great UNK , and but at any UNK he might UNK UNK -- how UNK i to get in ? ' she it UNK an UNK UNK , no UNK , and very UNK and UNK or UNK , his UNK and UNK -- that is , if i can UNK them . ' as she said this , she UNK it UNK all the UNK and -- oh dear ! ' cried alice in a UNK UNK that it was a very UNK UNK UNK . the UNK . UNK your UNK ! ' said the queen , UNK UNK . UNK your UNK , ' said the UNK , ' i did n't UNK it , and they 'that 's the UNK they 're called UNK , ' the gryphon UNK : 'but about his UNK ? ' the mock turtle UNK . UNK could he UNK them back to UNK his UNK . does n't like UNK so UNK , you know . ' UNK that UNK , ' said the dodo UNK , UNK to its feet , ' i UNK UNK upon a time there were three little UNK , ' the dormouse began UNK we UNK another UNK of the UNK UNK ? ' the gryphon went UNK for UNK of their UNK her ; and when she next UNK out the back to the UNK , the queen UNK UNK that a moment 's UNK would seemed to be no sort of 

In [22]:
test_text = replace_oov_in_text(test_text)
test_text

"UNK , the UNK UNK UNK her UNK UNK the UNK UNK of UNK n't it UNK ? ' said alice in a tone of great UNK . UNK without a moment 's UNK . the only things in the UNK UNK UNK UNK . however , she did not like to be UNK , so she she felt a little UNK about this ; UNK it might UNK , you know , ' said of UNK , ' i do wish they would put their UNK down ! i UNK so very UNK UNK . was a table , with a large UNK of UNK upon it : they looked so good , UNK mouse UNK his way through the UNK UNK -- she 'that 's very UNK , ' the king said , UNK to the jury . they were 'you 're UNK about the UNK , ' said the mock turtle : UNK would all UNK UNK the first day , ' said the mock turtle : UNK the next , and so like a UNK in the UNK . added the gryphon ; and then they UNK UNK UNK and looked at poor ' -- UNK , that 's about the right UNK -- but then i wonder what UNK little UNK UNK in the UNK , and to her great UNK it UNK ! could . that UNK to come once a UNK : he UNK UNK UNK , UNK , and first thought was that i

## Tokenization in sentences

Dividing a block of text into individual sentences.

In [23]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

sentences = sent_tokenize(train_text)

for sent in sentences:
    print(sent)
    print("_________________")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'if you UNK time as well as i do , ' said the hatter , 'you would n't UNK UNK .
_________________
UNK quite so much ! '
_________________
of things -- i ca n't UNK half of them -- and it UNK to a UNK , before she made her next UNK .
_________________
UNK the UNK day must have been a little thing UNK in UNK ( it had UNK off UNK by this time ) .
_________________
UNK with one UNK , as he found it made no UNK ; but he now hastily over all the UNK she UNK that were of the same UNK as herself , to you must UNK the next UNK .
_________________
it quite UNK my UNK and was just going to UNK the UNK , when her UNK UNK upon a little of UNK , UNK , UNK , UNK UNK , UNK , and UNK UNK of the gryphon , and all the other UNK UNK , would UNK ( she UNK in UNK .
_________________
UNK 'd soon UNK it back ! '
_________________
the UNK , the king , and the queen , who were all talking at once , but the UNK replied `` too UNK , too UNK ! ''
_________________
and gave a look UNK -- began UNK the sort of thing

## Tokenization (word level)

NLTK offers various methods for splitting your text into individual words or tokens. In this case, we will utilize the TweetTokenizer for this purpose.

In [24]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer

tweet_wt = TweetTokenizer()

sentences_tokenized = []

for sent in sentences:
    sent_tok = tweet_wt.tokenize(sent)
    sentences_tokenized.append(sent_tok)
    print(sent_tok)
    print("_________________")


["'", 'if', 'you', 'UNK', 'time', 'as', 'well', 'as', 'i', 'do', ',', "'", 'said', 'the', 'hatter', ',', "'", 'you', 'would', "n't", 'UNK', 'UNK', '.']
_________________
['UNK', 'quite', 'so', 'much', '!', "'"]
_________________
['of', 'things', '-', '-', 'i', 'ca', "n't", 'UNK', 'half', 'of', 'them', '-', '-', 'and', 'it', 'UNK', 'to', 'a', 'UNK', ',', 'before', 'she', 'made', 'her', 'next', 'UNK', '.']
_________________
['UNK', 'the', 'UNK', 'day', 'must', 'have', 'been', 'a', 'little', 'thing', 'UNK', 'in', 'UNK', '(', 'it', 'had', 'UNK', 'off', 'UNK', 'by', 'this', 'time', ')', '.']
_________________
['UNK', 'with', 'one', 'UNK', ',', 'as', 'he', 'found', 'it', 'made', 'no', 'UNK', ';', 'but', 'he', 'now', 'hastily', 'over', 'all', 'the', 'UNK', 'she', 'UNK', 'that', 'were', 'of', 'the', 'same', 'UNK', 'as', 'herself', ',', 'to', 'you', 'must', 'UNK', 'the', 'next', 'UNK', '.']
_________________
['it', 'quite', 'UNK', 'my', 'UNK', 'and', 'was', 'just', 'going', 'to', 'UNK', 'the', 

In [25]:
from pprint import pprint

tokens = tweet_wt.tokenize(train_text)
tokens

["'",
 'if',
 'you',
 'UNK',
 'time',
 'as',
 'well',
 'as',
 'i',
 'do',
 ',',
 "'",
 'said',
 'the',
 'hatter',
 ',',
 "'",
 'you',
 'would',
 "n't",
 'UNK',
 'UNK',
 '.',
 'UNK',
 'quite',
 'so',
 'much',
 '!',
 "'",
 'of',
 'things',
 '-',
 '-',
 'i',
 'ca',
 "n't",
 'UNK',
 'half',
 'of',
 'them',
 '-',
 '-',
 'and',
 'it',
 'UNK',
 'to',
 'a',
 'UNK',
 ',',
 'before',
 'she',
 'made',
 'her',
 'next',
 'UNK',
 '.',
 'UNK',
 'the',
 'UNK',
 'day',
 'must',
 'have',
 'been',
 'a',
 'little',
 'thing',
 'UNK',
 'in',
 'UNK',
 '(',
 'it',
 'had',
 'UNK',
 'off',
 'UNK',
 'by',
 'this',
 'time',
 ')',
 '.',
 'UNK',
 'with',
 'one',
 'UNK',
 ',',
 'as',
 'he',
 'found',
 'it',
 'made',
 'no',
 'UNK',
 ';',
 'but',
 'he',
 'now',
 'hastily',
 'over',
 'all',
 'the',
 'UNK',
 'she',
 'UNK',
 'that',
 'were',
 'of',
 'the',
 'same',
 'UNK',
 'as',
 'herself',
 ',',
 'to',
 'you',
 'must',
 'UNK',
 'the',
 'next',
 'UNK',
 '.',
 'it',
 'quite',
 'UNK',
 'my',
 'UNK',
 'and',
 'was',
 'just

## Tokens Frequency

Token frequency simply counts how often individual words or phrases appear in a text or dataset.

In [26]:
# Frequency distribution
N = 100
count = nltk.FreqDist(tokens)
print('{} most common tokens: \n')

pprint(count.most_common(N))

{} most common tokens: 

[('UNK', 4791),
 (',', 1656),
 ("'", 1522),
 ('the', 1135),
 ('.', 682),
 ('and', 610),
 ('to', 495),
 ('a', 423),
 ('it', 419),
 ('she', 389),
 ('of', 360),
 ('-', 358),
 ('i', 356),
 ('said', 307),
 ('!', 300),
 ('you', 267),
 ('alice', 267),
 ('in', 265),
 ('was', 255),
 ('that', 211),
 ('as', 177),
 ('her', 164),
 (':', 162),
 ('at', 144),
 (';', 137),
 ("n't", 136),
 ('?', 135),
 ('s', 132),
 ('had', 126),
 ('but', 125),
 ('all', 125),
 ('on', 121),
 ('with', 114),
 ('be', 107),
 ('for', 101),
 ('very', 98),
 ('not', 96),
 ('this', 93),
 ('what', 92),
 ('they', 90),
 ('so', 89),
 ('he', 89),
 ('do', 87),
 ('little', 85),
 ('out', 82),
 ('is', 79),
 ('down', 74),
 ('up', 73),
 ('one', 68),
 ('them', 67),
 ('if', 66),
 ('there', 65),
 ('would', 64),
 ('about', 64),
 ('know', 63),
 ('`', 62),
 ('his', 62),
 ('have', 61),
 ('could', 61),
 ('again', 60),
 ('like', 60),
 ('when', 57),
 ('were', 56),
 ('herself', 55),
 ('went', 55),
 ('did', 55),
 ('thought', 53)

# Create and count n-grams frequency with zip function

The [zip function](https://www.w3schools.com/python/ref_func_zip.asp) in Python is used to combine two or more iterables (e.g., lists, tuples) element-wise into tuples. When used to create n-grams, you can use it to pair words or tokens from a text to form n-grams by sliding a "window" of a specific size over the sequence

In [27]:
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in sentences_tokenized:

    # Update the unigram counter
    unigram_counter.update([(gram,) for gram in sent])

    # Update the bigram counter
    bigram_pad_sent = ["<start>"] + sent +  ["<end>"]
    bigram_counter.update([(gram1, gram2) for gram1, gram2 in zip(bigram_pad_sent, bigram_pad_sent[1:])])

    # Update the trigram counter
    trigram_pad_sent = ["<start1>", "<start2>"] + sent + ["<end>"]
    trigram_counter.update([(gram1, gram2, gram3) for gram1, gram2, gram3 in zip(trigram_pad_sent, trigram_pad_sent[1:], trigram_pad_sent[2:])])


pprint(unigram_counter.most_common(10))
pprint(bigram_counter.most_common(10))
pprint(trigram_counter.most_common(10))

[(('UNK',), 4791),
 ((',',), 1656),
 (("'",), 1522),
 (('the',), 1135),
 (('.',), 682),
 (('and',), 610),
 (('to',), 495),
 (('a',), 423),
 (('it',), 419),
 (('she',), 389)]
[(('UNK', ','), 733),
 (('UNK', 'UNK'), 702),
 (('the', 'UNK'), 500),
 (('.', '<end>'), 497),
 (("'", '<end>'), 481),
 ((',', "'"), 352),
 (('UNK', '.'), 310),
 ((',', 'and'), 298),
 (('<start>', 'UNK'), 276),
 ((',', 'UNK'), 249)]
[(('<start1>', '<start2>', 'UNK'), 276),
 (('UNK', '.', '<end>'), 215),
 (('!', "'", '<end>'), 187),
 (('.', "'", '<end>'), 169),
 (('UNK', ',', 'and'), 156),
 (('UNK', ',', "'"), 149),
 ((',', "'", 'said'), 144),
 (('UNK', 'UNK', ','), 129),
 (('UNK', ',', 'UNK'), 117),
 (('?', "'", '<end>'), 106)]


#  Calculate bi-gram probability

### $ P(w_2|w_1) = \frac{C(w_1,w_2) + \alpha}{C(w_1) + \alpha \cdot|V|} $

* $ C(w_1,w_2) $ : bigram count
* $ C(w_1) $ : unigram count
* $ 0 \leq\alpha \leq1 $ :  smoothing hyper-parameter
* |V|: vocabulary size

In [28]:
# We should fine-tune alpha on a held-out dataset
alpha = 0.01

# Calculate vocab size
# vocab_size = len(set(tokens))
vocab_size = len(vocab)

In [29]:
vocab_size

271

In [30]:
# Bigram prob + laplace smoothing
bigram_prob = (bigram_counter[('the', 'evening')] + alpha) / (unigram_counter[('the')] + alpha*vocab_size)
print("bigram_prob: {0:.3f} ".format(bigram_prob))

bigram_log_prob = math.log2(bigram_prob)
print("bigram_log_prob: {0:.3f}".format(bigram_log_prob) )

bigram_prob: 0.004 
bigram_log_prob: -8.082


#  Calculate tri-gram probability

### $ P(w_3|w_1,w_2) = \frac{C(w_1,w_2,w_3) + \alpha}{C(w_1,w_2) + \alpha \cdot|V|} $
Where:

$ C(w_1, w_2, w_3) $: Trigram count, the number of times the trigram (w1, w2, w3) appears in the corpus.
$ C(w_1, w_2) $: Bigram count, the number of times the bigram (w1, w2) appears in the corpus.
$ 0 \leq \alpha \leq 1 $: Smoothing hyper-parameter.
$ |V| $: Vocabulary size, the total number of unique words in the corpus.

In [31]:
# Triagram prob + laplace smoothing
triagram_prob = (trigram_counter[('the', 'evening','is')] + alpha) / (bigram_counter[('the','evening')] + alpha*vocab_size)
print("triagram_prob: {0:.3f} ".format(bigram_prob))

triagram_log_prob = math.log2(triagram_prob)
print("triagram_log_prob: {0:.3f}".format(triagram_log_prob) )

triagram_prob: 0.004 
triagram_log_prob: -8.082


## Test subset

We have to tokenize our test subset in tokens.

In [32]:
test_text

"UNK , the UNK UNK UNK her UNK UNK the UNK UNK of UNK n't it UNK ? ' said alice in a tone of great UNK . UNK without a moment 's UNK . the only things in the UNK UNK UNK UNK . however , she did not like to be UNK , so she she felt a little UNK about this ; UNK it might UNK , you know , ' said of UNK , ' i do wish they would put their UNK down ! i UNK so very UNK UNK . was a table , with a large UNK of UNK upon it : they looked so good , UNK mouse UNK his way through the UNK UNK -- she 'that 's very UNK , ' the king said , UNK to the jury . they were 'you 're UNK about the UNK , ' said the mock turtle : UNK would all UNK UNK the first day , ' said the mock turtle : UNK the next , and so like a UNK in the UNK . added the gryphon ; and then they UNK UNK UNK and looked at poor ' -- UNK , that 's about the right UNK -- but then i wonder what UNK little UNK UNK in the UNK , and to her great UNK it UNK ! could . that UNK to come once a UNK : he UNK UNK UNK , UNK , and first thought was that i

In [33]:
test_sentences = sent_tokenize(test_text)

for sent in test_sentences:
    print(sent)
    print("_________________")

UNK , the UNK UNK UNK her UNK UNK the UNK UNK of UNK n't it UNK ? '
_________________
said alice in a tone of great UNK .
_________________
UNK without a moment 's UNK .
_________________
the only things in the UNK UNK UNK UNK .
_________________
however , she did not like to be UNK , so she she felt a little UNK about this ; UNK it might UNK , you know , ' said of UNK , ' i do wish they would put their UNK down !
_________________
i UNK so very UNK UNK .
_________________
was a table , with a large UNK of UNK upon it : they looked so good , UNK mouse UNK his way through the UNK UNK -- she 'that 's very UNK , ' the king said , UNK to the jury .
_________________
they were 'you 're UNK about the UNK , ' said the mock turtle : UNK would all UNK UNK the first day , ' said the mock turtle : UNK the next , and so like a UNK in the UNK .
_________________
added the gryphon ; and then they UNK UNK UNK and looked at poor ' -- UNK , that 's about the right UNK -- but then i wonder what UNK litt

In [34]:
from nltk import word_tokenize
from nltk import WhitespaceTokenizer
from nltk.tokenize import TweetTokenizer

tweet_wt = TweetTokenizer()

test_sentences_tokenized = []

for sent in test_sentences:
    sent_tok = tweet_wt.tokenize(sent)
    test_sentences_tokenized.append(sent_tok)
    print(sent_tok)
    print("_________________")


['UNK', ',', 'the', 'UNK', 'UNK', 'UNK', 'her', 'UNK', 'UNK', 'the', 'UNK', 'UNK', 'of', 'UNK', "n't", 'it', 'UNK', '?', "'"]
_________________
['said', 'alice', 'in', 'a', 'tone', 'of', 'great', 'UNK', '.']
_________________
['UNK', 'without', 'a', 'moment', "'", 's', 'UNK', '.']
_________________
['the', 'only', 'things', 'in', 'the', 'UNK', 'UNK', 'UNK', 'UNK', '.']
_________________
['however', ',', 'she', 'did', 'not', 'like', 'to', 'be', 'UNK', ',', 'so', 'she', 'she', 'felt', 'a', 'little', 'UNK', 'about', 'this', ';', 'UNK', 'it', 'might', 'UNK', ',', 'you', 'know', ',', "'", 'said', 'of', 'UNK', ',', "'", 'i', 'do', 'wish', 'they', 'would', 'put', 'their', 'UNK', 'down', '!']
_________________
['i', 'UNK', 'so', 'very', 'UNK', 'UNK', '.']
_________________
['was', 'a', 'table', ',', 'with', 'a', 'large', 'UNK', 'of', 'UNK', 'upon', 'it', ':', 'they', 'looked', 'so', 'good', ',', 'UNK', 'mouse', 'UNK', 'his', 'way', 'through', 'the', 'UNK', 'UNK', '-', '-', 'she', "'", 'that', 

In [35]:
len(test_sentences_tokenized)

263

## Bi-gram LM Cross entropy & perplexity

* $ CrossEntropy = -\frac{1}{N}\sum^{bigrams}{log_2(P(w_2|w_1))} $

* N: Number of bigrams
* $ Perplexity = 2^{H(p)} $

In [36]:
from itertools import pairwise

sum_prob = 0
bigram_cnt = 0

for sent in sentences_tokenized:
    sent = ['<start>'] + sent + ['<end>']

    # Iterate over the bigrmas of the sentence
    for first_token, second_token in pairwise(sent):
        bigram_prob = (bigram_counter[(first_token, second_token)] + alpha) / (unigram_counter[(first_token,)] + alpha*vocab_size)
        sum_prob += math.log2(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
bi_perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(bi_perpl))

Cross Entropy: 3.660
perplexity: 12.645


In [37]:
# list(pairwise(sent))

## Tri-gram LM Cross entropy & perplexity

* $ CrossEntropy = -\frac{1}{N}\sum_{trigrams} log_2(P(w_3|w_1, w_2)) $
* N: Number of bigrams
* $ Perplexity = 2^{H(p)} $

In [38]:
from more_itertools import windowed

sum_prob = 0
trigram_cnt = 0

for sent in sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    for first_token, second_token, third_token in windowed(sent, n=3):
        trigram_prob = (trigram_counter[(first_token, second_token, third_token)] + alpha) / (bigram_counter[(first_token, second_token)] + alpha*vocab_size)
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
tri_perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(tri_perpl))

Cross Entropy: 2.417
perplexity: 5.342


## Interpolated bi-gram and tri-gram LM
### $ P(w_3|w_1,w_2) = \lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2)  $
* $ 0 \leq \lambda \leq 1 $
* $ \lambda $ should be fine-tuned on a held-out dataset
---
$ CrossEntropy = -\frac{1}{N}\sum^{trigrmas}{log_2(P(w_3|w_1,w_2))} = -\frac{1}{N}\sum^{trigrmas}{log_2(\lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2))}$
* N: Number of trigrams <br>

$ Perplexity = 2^{H(p)} $

In [39]:
lamda = 0.0001
sum_prob = 0
ngram_cnt = 0

for sent in sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    for first_token, second_token, third_token in windowed(sent, n=3):

        trigram_prob = (trigram_counter[(first_token, second_token, third_token)] + alpha) / (bigram_counter[(first_token, second_token)] + alpha*vocab_size)
        bigram_prob = (bigram_counter[(second_token, third_token)] + alpha) / (unigram_counter[(second_token,)] + alpha*vocab_size)

        sum_prob += math.log2(lamda * trigram_prob + (1-lamda) * bigram_prob)
        ngram_cnt += 1

        # print('P({}| {}, {})'.format(sent[idx],sent[idx-2], sent[idx-1]))
        # print('P({}| {})'.format(sent[idx], sent[idx-1]))

HC = -sum_prob / ngram_cnt
int_perpl = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(int_perpl))

Cross Entropy: 4.150
perplexity: 17.752


In [40]:
data = [
    ["Bi-gram Perplexity", bi_perpl],
    ["Tri-gram Perplexity", tri_perpl],
    ["Interpolation Perplexity", int_perpl],
]

# Create the table
table = tabulate(data, headers=["Variable" , "Value"], tablefmt="grid")

# Print the table
print(table)

+--------------------------+----------+
| Variable                 |    Value |
+==========================+==========+
| Bi-gram Perplexity       | 12.6448  |
+--------------------------+----------+
| Tri-gram Perplexity      |  5.34222 |
+--------------------------+----------+
| Interpolation Perplexity | 17.7515  |
+--------------------------+----------+


As we can observe, the tri-gram model has the lowest perplexity. Lower perplexity values indicate that the model is better at predicting the data

# Hyperparameters Tuning

Tuning is necessary to determine the optimal alpha and lambda values by selecting the settings that provide the best model performance. We are going to use our validation test to do so.

In [41]:
valid_text

"'oh , i UNK your UNK ! ' she UNK in a tone of great UNK , and but at any UNK he might UNK UNK -- how UNK i to get in ? ' she it UNK an UNK UNK , no UNK , and very UNK and UNK or UNK , his UNK and UNK -- that is , if i can UNK them . ' as she said this , she UNK it UNK all the UNK and -- oh dear ! ' cried alice in a UNK UNK that it was a very UNK UNK UNK . the UNK . UNK your UNK ! ' said the queen , UNK UNK . UNK your UNK , ' said the UNK , ' i did n't UNK it , and they 'that 's the UNK they 're called UNK , ' the gryphon UNK : 'but about his UNK ? ' the mock turtle UNK . UNK could he UNK them back to UNK his UNK . does n't like UNK so UNK , you know . ' UNK that UNK , ' said the dodo UNK , UNK to its feet , ' i UNK UNK upon a time there were three little UNK , ' the dormouse began UNK we UNK another UNK of the UNK UNK ? ' the gryphon went UNK for UNK of their UNK her ; and when she next UNK out the back to the UNK , the queen UNK UNK that a moment 's UNK would seemed to be no sort of 

In [42]:
tweet_wt = TweetTokenizer()

valid_sentences = sent_tokenize(valid_text)
for sent in valid_sentences:
    print(sent)
    print("_________________")

'oh , i UNK your UNK ! '
_________________
she UNK in a tone of great UNK , and but at any UNK he might UNK UNK -- how UNK i to get in ? '
_________________
she it UNK an UNK UNK , no UNK , and very UNK and UNK or UNK , his UNK and UNK -- that is , if i can UNK them . '
_________________
as she said this , she UNK it UNK all the UNK and -- oh dear ! '
_________________
cried alice in a UNK UNK that it was a very UNK UNK UNK .
_________________
the UNK .
_________________
UNK your UNK ! '
_________________
said the queen , UNK UNK .
_________________
UNK your UNK , ' said the UNK , ' i did n't UNK it , and they 'that 's the UNK they 're called UNK , ' the gryphon UNK : 'but about his UNK ? '
_________________
the mock turtle UNK .
_________________
UNK could he UNK them back to UNK his UNK .
_________________
does n't like UNK so UNK , you know . '
_________________
UNK that UNK , ' said the dodo UNK , UNK to its feet , ' i UNK UNK upon a time there were three little UNK , ' the dormous

In [43]:
valid_text_tokenized = []
# valid_sentences = sent_tokenize(valid_text)

for sent in valid_sentences:
    sent_tok = tweet_wt.tokenize(sent)
    valid_text_tokenized.append(sent_tok)
    print(sent_tok)
    print("_________________")

["'", 'oh', ',', 'i', 'UNK', 'your', 'UNK', '!', "'"]
_________________
['she', 'UNK', 'in', 'a', 'tone', 'of', 'great', 'UNK', ',', 'and', 'but', 'at', 'any', 'UNK', 'he', 'might', 'UNK', 'UNK', '-', '-', 'how', 'UNK', 'i', 'to', 'get', 'in', '?', "'"]
_________________
['she', 'it', 'UNK', 'an', 'UNK', 'UNK', ',', 'no', 'UNK', ',', 'and', 'very', 'UNK', 'and', 'UNK', 'or', 'UNK', ',', 'his', 'UNK', 'and', 'UNK', '-', '-', 'that', 'is', ',', 'if', 'i', 'can', 'UNK', 'them', '.', "'"]
_________________
['as', 'she', 'said', 'this', ',', 'she', 'UNK', 'it', 'UNK', 'all', 'the', 'UNK', 'and', '-', '-', 'oh', 'dear', '!', "'"]
_________________
['cried', 'alice', 'in', 'a', 'UNK', 'UNK', 'that', 'it', 'was', 'a', 'very', 'UNK', 'UNK', 'UNK', '.']
_________________
['the', 'UNK', '.']
_________________
['UNK', 'your', 'UNK', '!', "'"]
_________________
['said', 'the', 'queen', ',', 'UNK', 'UNK', '.']
_________________
['UNK', 'your', 'UNK', ',', "'", 'said', 'the', 'UNK', ',', "'", 'i', 'd

In [44]:
valid_text_tokenized

[["'", 'oh', ',', 'i', 'UNK', 'your', 'UNK', '!', "'"],
 ['she',
  'UNK',
  'in',
  'a',
  'tone',
  'of',
  'great',
  'UNK',
  ',',
  'and',
  'but',
  'at',
  'any',
  'UNK',
  'he',
  'might',
  'UNK',
  'UNK',
  '-',
  '-',
  'how',
  'UNK',
  'i',
  'to',
  'get',
  'in',
  '?',
  "'"],
 ['she',
  'it',
  'UNK',
  'an',
  'UNK',
  'UNK',
  ',',
  'no',
  'UNK',
  ',',
  'and',
  'very',
  'UNK',
  'and',
  'UNK',
  'or',
  'UNK',
  ',',
  'his',
  'UNK',
  'and',
  'UNK',
  '-',
  '-',
  'that',
  'is',
  ',',
  'if',
  'i',
  'can',
  'UNK',
  'them',
  '.',
  "'"],
 ['as',
  'she',
  'said',
  'this',
  ',',
  'she',
  'UNK',
  'it',
  'UNK',
  'all',
  'the',
  'UNK',
  'and',
  '-',
  '-',
  'oh',
  'dear',
  '!',
  "'"],
 ['cried',
  'alice',
  'in',
  'a',
  'UNK',
  'UNK',
  'that',
  'it',
  'was',
  'a',
  'very',
  'UNK',
  'UNK',
  'UNK',
  '.'],
 ['the', 'UNK', '.'],
 ['UNK', 'your', 'UNK', '!', "'"],
 ['said', 'the', 'queen', ',', 'UNK', 'UNK', '.'],
 ['UNK',
  'your

## Alpha Value Tuning

We train the model on a training subset with the current alpha and calculates cross-entropy on a validation subset for each alpha value. The goal is to identify the alpha value that results in the lowest cross-entropy, indicating the best model performance. The selected "best_alpha" is the one that minimizes the cross-entropy, making it a suitable hyperparameter for the language model.

In [45]:
# Define the range of alpha values to test
alpha_values = [0.1, 0.01, 0.001, 0.0001]

best_alpha = None
best_cross_entropy = float('inf')

exclude_start = True  # Set this flag to True to exclude "<start>" tokens

for alpha in alpha_values:

    sum_prob = 0
    bigram_cnt = 0

    for sent in valid_text_tokenized:
        sent = ['<start>'] + sent + ['<end>']

        for idx in range(1, len(sent)):
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha * vocab_size)
            sum_prob += math.log(bigram_prob)
            bigram_cnt += 1

    HC = -sum_prob / bigram_cnt

    if HC < best_cross_entropy:
        best_cross_entropy = HC
        best_alpha = alpha

# Print the best hyperparameter value
print("Best Alpha:", best_alpha)


Best Alpha: 0.01


## Lambda Value Tuning

We follow the same procedure for the lambda value

In [46]:
# Define the range of lambda values to test
lambda_values = [0.1, 0.01, 0.001, 0.0001]

best_lambda = None
best_cross_entropy = float('inf')

exclude_start = True  # Set this flag to True to exclude "<start1>" and "<start2>" tokens

for lambd in lambda_values:
    # Train the model with the current lambda value on the training subset
    # Calculate interpolated bigram probabilities and cross-entropy on the validation subset
    # Adjust the following code according to your specific language modeling approach

    sum_prob = 0
    bigram_cnt = 0

    for sent in valid_text_tokenized:
        sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

        for idx in range(1, len(sent)):
            # if sent[idx] in ['<start1>', '<start2>'] and exclude_start:
            #     continue
            # Calculate interpolated bigram probability
            trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] +alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] +alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)

            sum_prob += math.log(bigram_prob)  # Use math.log instead of math.log2
            bigram_cnt += 1

    HC = -sum_prob / bigram_cnt

    if HC < best_cross_entropy:
        best_cross_entropy = HC
        best_lambda = lambd

# Print the best lambda value
print("Best Lambda:", best_lambda)


Best Lambda: 0.1


# After tuning

With the tuned hyperparameters, we can expect improved model performance and better predictive accuracy.

## Bi-gram LM Cross entropy & perplexity

In [47]:
alpha = best_alpha
sum_prob = 0
bigram_cnt = 0


for sent in test_sentences_tokenized:
    sent = ['<start>'] + sent + ['<end']

    # Iterate over the bigrams of the sentence
    for idx in range(1, len(sent)):
        bigram_prob = (bigram_counter[(sent[idx - 1], sent[idx])] + alpha) / (unigram_counter[(sent[idx - 1],)] + alpha * vocab_size)
        sum_prob += math.log(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
bi_perpl1 = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("Perplexity: {0:.3f}".format(bi_perpl1))


Cross Entropy: 3.457
Perplexity: 10.983


In [48]:
bigram_counter

Counter({('<start>', "'"): 100,
         ("'", 'if'): 13,
         ('if', 'you'): 18,
         ('you', 'UNK'): 48,
         ('UNK', 'time'): 3,
         ('time', 'as'): 2,
         ('as', 'well'): 13,
         ('well', 'as'): 5,
         ('as', 'i'): 3,
         ('i', 'do'): 27,
         ('do', ','): 7,
         (',', "'"): 352,
         ("'", 'said'): 147,
         ('said', 'the'): 142,
         ('the', 'hatter'): 36,
         ('hatter', ','): 8,
         ("'", 'you'): 41,
         ('you', 'would'): 6,
         ('would', "n't"): 7,
         ("n't", 'UNK'): 52,
         ('UNK', 'UNK'): 702,
         ('UNK', '.'): 310,
         ('.', '<end>'): 497,
         ('<start>', 'UNK'): 276,
         ('UNK', 'quite'): 6,
         ('quite', 'so'): 1,
         ('so', 'much'): 4,
         ('much', '!'): 1,
         ('!', "'"): 196,
         ("'", '<end>'): 481,
         ('<start>', 'of'): 5,
         ('of', 'things'): 3,
         ('things', '-'): 2,
         ('-', '-'): 179,
         ('-', 'i'): 6,


## Tri-gram LM Cross entropy & perplexity

In [49]:
alpha = best_alpha
sum_prob = 0
trigram_cnt = 0

for sent in test_sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end']

    for idx in range(2, len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx - 2], sent[idx - 1], sent[idx])] + alpha) / (bigram_counter[(sent[idx - 2], sent[idx - 1])] + alpha * vocab_size)
        sum_prob += math.log(trigram_prob)
        trigram_cnt += 1

HC = -sum_prob / trigram_cnt
tri_perpl1 = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("Perplexity: {0:.3f}".format(tri_perpl1))


Cross Entropy: 3.376
Perplexity: 10.385


## Interpolated bi-gram and tri-gram LM

In [50]:
alpha = best_alpha
lamda = best_lambda
sum_prob = 0
ngram_cnt = 0
exclude_start = True

for sent in test_sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    for idx in range(2, len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx-2], sent[idx-1], sent[idx])] + alpha) / (bigram_counter[(sent[idx-2], sent[idx-1])] + alpha * vocab_size)
        bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha * vocab_size)

        sum_prob += math.log(lamda * trigram_prob + (1 - lamda) * bigram_prob)
        ngram_cnt += 1

HC = -sum_prob / ngram_cnt
int_perpl1 = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(int_perpl1))


Cross Entropy: 3.039
perplexity: 8.217


In [51]:
data = [
    ["Bi-gram Perplexity",bi_perpl ,bi_perpl1],
    ["Tri-gram Perplexity",tri_perpl, tri_perpl1],
    ["Interpolation Perplexity", int_perpl,int_perpl1],
]

# Create the table
table = tabulate(data, headers=["Before Tuning","After Tuning"], tablefmt="grid")

# Print the table
print(table)

+--------------------------+-----------------+----------------+
|                          |   Before Tuning |   After Tuning |
+==========================+=================+================+
| Bi-gram Perplexity       |        12.6448  |        10.9831 |
+--------------------------+-----------------+----------------+
| Tri-gram Perplexity      |         5.34222 |        10.3852 |
+--------------------------+-----------------+----------------+
| Interpolation Perplexity |        17.7515  |         8.2166 |
+--------------------------+-----------------+----------------+


Based on the table provided, it is evident that our most effective model is the untuned trigram model.

# Generate sentence completions
We are going to produce sentence completions by choosing the most likely word at each step until the generated sentence concludes with the end token.

## Bigram Model

As evident, our models contain unidentified tokens (UNKs), which pose a challenge when predicting certain words. Furthermore, it's noteworthy that UNK is the most prevalent token. Consequently, we have opted to eliminate all UNKs from our models.

In [52]:
# Initialize a new bigram counter without bigrams containing "UNK"
filtered_bigram_counter = {}

for bigram, count in bigram_counter.items():
    # Check if "UNK" is not in any of the words in the bigram
    if 'UNK' not in bigram:
        filtered_bigram_counter[bigram] = count

# Now, filtered_bigram_counter contains only the bigrams without "UNK"
filtered_bigram_counter

{('<start>', "'"): 100,
 ("'", 'if'): 13,
 ('if', 'you'): 18,
 ('time', 'as'): 2,
 ('as', 'well'): 13,
 ('well', 'as'): 5,
 ('as', 'i'): 3,
 ('i', 'do'): 27,
 ('do', ','): 7,
 (',', "'"): 352,
 ("'", 'said'): 147,
 ('said', 'the'): 142,
 ('the', 'hatter'): 36,
 ('hatter', ','): 8,
 ("'", 'you'): 41,
 ('you', 'would'): 6,
 ('would', "n't"): 7,
 ('.', '<end>'): 497,
 ('quite', 'so'): 1,
 ('so', 'much'): 4,
 ('much', '!'): 1,
 ('!', "'"): 196,
 ("'", '<end>'): 481,
 ('<start>', 'of'): 5,
 ('of', 'things'): 3,
 ('things', '-'): 2,
 ('-', '-'): 179,
 ('-', 'i'): 6,
 ('i', 'ca'): 10,
 ('ca', "n't"): 17,
 ('half', 'of'): 3,
 ('of', 'them'): 15,
 ('them', '-'): 3,
 ('-', 'and'): 18,
 ('and', 'it'): 12,
 ('to', 'a'): 7,
 (',', 'before'): 3,
 ('before', 'she'): 9,
 ('she', 'made'): 5,
 ('made', 'her'): 3,
 ('her', 'next'): 1,
 ('day', 'must'): 1,
 ('must', 'have'): 8,
 ('have', 'been'): 7,
 ('been', 'a'): 2,
 ('a', 'little'): 34,
 ('little', 'thing'): 6,
 ('(', 'it'): 3,
 ('it', 'had'): 10,
 ('b

In [53]:
def bi_generate_sentence(initial_phrase):
    # Check if initial_phrase is a unigram
    if initial_phrase in filtered_bigram_counter:
        initial_words = [initial_phrase]
    else:
        initial_words = initial_phrase.split()
    last_word = initial_words[-1]

    if len(initial_words) < 2:
        # Find the bigram with the highest probability in the filtered_bigram_counter
        # Use argmax to select the bigram with the highest probability
        argmax_bigram = max(filtered_bigram_counter, key=filtered_bigram_counter.get)
        initial_words += argmax_bigram

    # Generate the sentence until "<end>" is encountered
    while True:
        # Find possible bigrams starting with the last word
        possible_bigrams = [bigram for bigram in filtered_bigram_counter if bigram[0] == last_word]

        if not possible_bigrams:
            break  # If no valid bigrams, break the loop

        # Select a random bigram based on their counts
        # next_bigram = random.choices(possible_bigrams, [filtered_bigram_counter[bigram] for bigram in possible_bigrams])[0]
        # Use argmax to select the most probable bigram
        next_bigram = max(possible_bigrams, key=lambda bigram: filtered_bigram_counter[bigram])
        # print(next_bigram)

        # Append the second word of the selected bigram to initial_words
        initial_words.append(next_bigram[1])

        # Update last_word for the next iteration
        last_word = next_bigram[1]

        # If the second word is '<end>', stop generating
        if next_bigram[1] == '<end>':
            break

    # Append '<end>' if it's not already present
    if '<end>' not in initial_words:
        initial_words.append('<end>')

    # Join the generated words into a complete sentence
    generated_sentence = ' '.join(initial_words)

    return generated_sentence


In [54]:
# Example usage
initial_phrase = 'this is'

generated_sentence = bi_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)


Generated sentence: this is , ' <end>


In [55]:
# Example usage
initial_phrase = 'this unknownword'


generated_sentence = bi_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)


Generated sentence: this unknownword <end>


## Trigram Model

In [56]:
# Initialize a new bigram counter without bigrams containing "UNK"
filtered_trigram_counter = {}

for trigram, count in trigram_counter.items():
    # Check if "UNK" is not in any of the words in the trigram
    if 'UNK' not in trigram:
        filtered_trigram_counter[trigram] = count

# Now, filtered_trigram_counter contains only the trigrams without "UNK"
filtered_trigram_counter

{('<start1>', '<start2>', "'"): 100,
 ('<start2>', "'", 'if'): 5,
 ("'", 'if', 'you'): 7,
 ('time', 'as', 'well'): 1,
 ('as', 'well', 'as'): 4,
 ('well', 'as', 'i'): 1,
 ('as', 'i', 'do'): 1,
 ('i', 'do', ','): 1,
 ('do', ',', "'"): 2,
 (',', "'", 'said'): 144,
 ("'", 'said', 'the'): 95,
 ('said', 'the', 'hatter'): 13,
 ('the', 'hatter', ','): 8,
 ('hatter', ',', "'"): 2,
 (',', "'", 'you'): 8,
 ("'", 'you', 'would'): 1,
 ('you', 'would', "n't"): 4,
 ('quite', 'so', 'much'): 1,
 ('so', 'much', '!'): 1,
 ('much', '!', "'"): 1,
 ('!', "'", '<end>'): 187,
 ('<start1>', '<start2>', 'of'): 5,
 ('<start2>', 'of', 'things'): 1,
 ('of', 'things', '-'): 1,
 ('things', '-', '-'): 2,
 ('-', '-', 'i'): 6,
 ('-', 'i', 'ca'): 1,
 ('i', 'ca', "n't"): 10,
 ('half', 'of', 'them'): 1,
 ('of', 'them', '-'): 1,
 ('them', '-', '-'): 3,
 ('-', '-', 'and'): 18,
 ('-', 'and', 'it'): 1,
 (',', 'before', 'she'): 1,
 ('before', 'she', 'made'): 1,
 ('she', 'made', 'her'): 1,
 ('made', 'her', 'next'): 1,
 ('day', 

In [57]:
def tri_generate_sentence(initial_phrase):
    initial_words = initial_phrase.split()  # Tokenize the initial phrase

    if (initial_words[1], initial_words[2]) not in filtered_bigram_counter:
          argmax_trigram = max(filtered_bigram_counter, key=filtered_bigram_counter.get)
          initial_words += argmax_trigram


    else:
        # Generate the sentence until "<end>" is encountered
        while True:
            # Get the last word in initial_words
            last_word = initial_words[-1]

            # Find possible bigrams starting with the last word
            possible_trigrams = [trigram for trigram in filtered_trigram_counter if trigram[0] == last_word]

            if not possible_trigrams:
                break  # If no valid trigrams, break the loop

            # Choose the trigram with the highest probatrility based on the two previous words
            next_trigram = max(possible_trigrams, key=lambda trigram: filtered_trigram_counter[trigram])
            # next_trigram =  random.choices(possible_trigrams, [filtered_trigram_counter[trigram] for trigram in possible_trigrams])[0]
            # Append the second word of the selected trigram to initial_words
            initial_words.append(next_trigram[1])

            # If the third word is '<end>', stop generating
            if next_trigram[-1] == '<end>':
              initial_words.append('<end>')
              break

        # Join the generated words into a complete sentence
    generated_sentence = ' '.join(initial_words)

    return generated_sentence


In [58]:
# Example usage
initial_phrase = 'she did not'
generated_sentence = tri_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)

Generated sentence: she did not , ' said alice . <end>


In [59]:
# Example usage
initial_phrase = 'this is unknownword'
generated_sentence = tri_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)

Generated sentence: this is unknownword . <end>



A trigram model is generally preferred over a bigram model for text generation due to its ability to consider a more extended context. While a bigram model looks at the preceding word when predicting the next word, a trigram model takes into account the two preceding words, which offers a richer and more detailed understanding of language.

## Models Comparison

In [60]:
filtered_bigram_counter

{('<start>', "'"): 100,
 ("'", 'if'): 13,
 ('if', 'you'): 18,
 ('time', 'as'): 2,
 ('as', 'well'): 13,
 ('well', 'as'): 5,
 ('as', 'i'): 3,
 ('i', 'do'): 27,
 ('do', ','): 7,
 (',', "'"): 352,
 ("'", 'said'): 147,
 ('said', 'the'): 142,
 ('the', 'hatter'): 36,
 ('hatter', ','): 8,
 ("'", 'you'): 41,
 ('you', 'would'): 6,
 ('would', "n't"): 7,
 ('.', '<end>'): 497,
 ('quite', 'so'): 1,
 ('so', 'much'): 4,
 ('much', '!'): 1,
 ('!', "'"): 196,
 ("'", '<end>'): 481,
 ('<start>', 'of'): 5,
 ('of', 'things'): 3,
 ('things', '-'): 2,
 ('-', '-'): 179,
 ('-', 'i'): 6,
 ('i', 'ca'): 10,
 ('ca', "n't"): 17,
 ('half', 'of'): 3,
 ('of', 'them'): 15,
 ('them', '-'): 3,
 ('-', 'and'): 18,
 ('and', 'it'): 12,
 ('to', 'a'): 7,
 (',', 'before'): 3,
 ('before', 'she'): 9,
 ('she', 'made'): 5,
 ('made', 'her'): 3,
 ('her', 'next'): 1,
 ('day', 'must'): 1,
 ('must', 'have'): 8,
 ('have', 'been'): 7,
 ('been', 'a'): 2,
 ('a', 'little'): 34,
 ('little', 'thing'): 6,
 ('(', 'it'): 3,
 ('it', 'had'): 10,
 ('b

In [61]:
filtered_trigram_counter

{('<start1>', '<start2>', "'"): 100,
 ('<start2>', "'", 'if'): 5,
 ("'", 'if', 'you'): 7,
 ('time', 'as', 'well'): 1,
 ('as', 'well', 'as'): 4,
 ('well', 'as', 'i'): 1,
 ('as', 'i', 'do'): 1,
 ('i', 'do', ','): 1,
 ('do', ',', "'"): 2,
 (',', "'", 'said'): 144,
 ("'", 'said', 'the'): 95,
 ('said', 'the', 'hatter'): 13,
 ('the', 'hatter', ','): 8,
 ('hatter', ',', "'"): 2,
 (',', "'", 'you'): 8,
 ("'", 'you', 'would'): 1,
 ('you', 'would', "n't"): 4,
 ('quite', 'so', 'much'): 1,
 ('so', 'much', '!'): 1,
 ('much', '!', "'"): 1,
 ('!', "'", '<end>'): 187,
 ('<start1>', '<start2>', 'of'): 5,
 ('<start2>', 'of', 'things'): 1,
 ('of', 'things', '-'): 1,
 ('things', '-', '-'): 2,
 ('-', '-', 'i'): 6,
 ('-', 'i', 'ca'): 1,
 ('i', 'ca', "n't"): 10,
 ('half', 'of', 'them'): 1,
 ('of', 'them', '-'): 1,
 ('them', '-', '-'): 3,
 ('-', '-', 'and'): 18,
 ('-', 'and', 'it'): 1,
 (',', 'before', 'she'): 1,
 ('before', 'she', 'made'): 1,
 ('she', 'made', 'her'): 1,
 ('made', 'her', 'next'): 1,
 ('day', 

We need to demonstrate, through the presentation of some sample generated texts, that the trigram model produces more coherent and fluent text compared to the bigram model.

In [62]:
initial_phrases = [
    'at the door',
    'did so indeed',
    'turn on the',
    'which seemed to',
    "the dormhouse half down"
]

In [63]:
bigram_sentences = []
trigram_sentences = []

In [64]:
for initial_phrase in initial_phrases:
    generated_sentence = bi_generate_sentence(initial_phrase)
    bigram_sentences.append(generated_sentence)

In [65]:
for initial_phrase in initial_phrases:
    generated_sentence = tri_generate_sentence(initial_phrase)
    trigram_sentences.append(generated_sentence)

In [66]:
table_data = list(zip(initial_phrases, bigram_sentences, trigram_sentences))
headers = ["Initial Phrase", "Bigram Sentence", "Trigram Sentence"]

In [67]:
table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

+-------------------------+------------------------------------+------------------------------------+
|     Initial Phrase      |          Bigram Sentence           |          Trigram Sentence          |
+-------------------------+------------------------------------+------------------------------------+
|       at the door       |       at the door , ' <end>        | at the door , ' said alice . <end> |
|      did so indeed      |        did so indeed <end>         |       did so indeed . <end>        |
|       turn on the       |     turn on the king , ' <end>     |  turn on the mock turtle . <end>   |
|     which seemed to     | which seemed to the king , ' <end> |  which seemed to herself . <end>   |
| the dormhouse half down | the dormhouse half down , ' <end>  |  the dormhouse half down . <end>   |
+-------------------------+------------------------------------+------------------------------------+


# Context - Aware Spelling Corrector

This Context - Aware Spelling Corrector will utilize my trigram language model, employ a beam search decoder for better candidate selection, and incorporate the mathematical formulas from [slide 19](https://eclass.aueb.gr/modules/document/file.php/INF210/slides_2023_24/nlp_slides_part01_ngrams_spelling_correction.pdf) to make accurate corrections.

Given a text input, you need to identify words that are potentially misspelled. This can be done using techniques like Levenshtein distance, phonetic similarity, or a dictionary lookup against a known list of correctly spelled words.

In [68]:
Levenshtein =  nltk.edit_distance

For each identified misspelled word, create a set of candidate corrections by applying common edit operations. These operations include:

*   Insertion: Add a single character at any position in the word.
*   Deletion: Remove a single character from the word.
*   Substitution: Replace a single character with another character.
*   Transposition: Swap adjacent characters in the word.

In [69]:
def generate_candidates(word, max_distance):
    # Generate candidates based on Levenshtein distance
    candidates = [candidate for candidate in vocab if Levenshtein(word, candidate) <= max_distance]
    return candidates

In [70]:
def score(candidate, word):
    levenshtein_score = -Levenshtein(candidate, word)
    trigram_score = filtered_trigram_counter.get((word, candidate), 0)
    return levenshtein_score + trigram_score

In [71]:
def beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score):
    words = initial_sentence.split()  # Split the sentence into words
    corrected_words = []

    for word in words:
        sequences = [([word], 0)]  # Initial sequence with a score of 0

        for _ in range(beam_width):  # Perform two iterations (you can adjust this)
            candidates = []
            for seq, seq_score in sequences:
                last_word = seq[-1]
                for next_word in generate_candidates(last_word, max_distance):
                    new_seq = seq + [next_word]
                    new_score = seq_score + score(next_word, last_word)
                    candidates.append((new_seq, new_score))

            # Select the top candidates based on score
            candidates.sort(key=lambda x: x[1], reverse=True)
            if candidates:
                sequences = candidates[:beam_width]
            else:
                # Handle the case when no candidates are available
                sequences = [([last_word], seq_score)]  # Revert to the last word as the only option

        best_correction = sequences[0][0][-1]  # Get the best correction for the word
        corrected_words.append(best_correction)

    corrected_sentence = " ".join(corrected_words)
    return corrected_sentence


In [72]:
# Misspelled sentence
initial_sentence = "ws cred mch"
max_distance = 5  # Maximum Levenshtein distance allowed
beam_width = 3  # Number of candidates to consider

# Find the best correction for the sentence
best_correction = beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score)

print("Corrected Sentence:", best_correction)

Corrected Sentence: as cried much


In [73]:
# Misspelled sentence
initial_sentence = "wts cred mch"
max_distance = 5  # Maximum Levenshtein distance allowed
beam_width = 2  # Number of candidates to consider

# Find the best correction for the sentence
best_correction = beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score)

print("Corrected Sentence:", best_correction)

Corrected Sentence: was cried much


In [74]:
# Misspelled sentence
initial_sentence = "mie tio"
max_distance = 5  # Maximum Levenshtein distance allowed
beam_width = 2  # Number of candidates to consider

# Find the best correction for the sentence
best_correction = beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score)

print("Corrected Sentence:", best_correction)

Corrected Sentence: me to


#  Evaluate the Context-Aware Spelling Corrector

The misspelling_probability is used to control how frequently misspellings are introduced into the sentences when creating the artificial test dataset. In the provided code, misspelling_probability is set to 0.05, which means there is a 5% chance that any given character in a sentence will be replaced with a misspelled character.

In [75]:
# Define the misspelling probability (e.g., 0.05 for 5%)
misspelling_probability = 0.05

In [76]:
replacement_rules = {
    'a': 'e',
    'b': 'v',
    'c': 's',
    'd': 't',
    'e': 'i',
    'f': 'g',
    'g': 'j',
    'h': 'n',
    'i': 'o',
    'j': 'k',
    'k': 'l',
    'l': 'm',
    'm': 'n',
    'n': 'm',
    'o': 'p',
    'p': 'o',
    'q': 'w',
    'r': 't',
    's': 'd',
    't': 'y',
    'u': 'y',
    'v': 'b',
    'w': 'q',
    'x': 'z',
    'y': 'u',
    'z': 'x',
    ',': '.',
    '.': ',',
    ' ': ' ',
}


In [77]:
def introduce_misspellings(sentence):
    misspelled_sentence = ''
    for char in sentence:
        if char != ' ' and random.random() <= misspelling_probability:
            if char == 'U' and sentence.startswith("UNK"):
                misspelled_sentence += 'y'
            else:
                misspelled_sentence += replacement_rules.get(char, char)
        else:
            misspelled_sentence += char
    return misspelled_sentence

In [78]:
test_sentences = [introduce_misspellings(sentence) for sentence in test_set]

In [79]:
misspelled_dataset = [introduce_misspellings(sentence) for sentence in test_sentences]

In [80]:
def correct_spelling_with_beam_search(misspelled_sentence, max_distance, beam_width, generate_candidates, score):
    max_distance = 5  # Maximum Levenshtein distance allowed
    beam_width = 2  # Number of candidates to consider
    corrected_sentence = beam_search_decode(misspelled_sentence, max_distance, beam_width, generate_candidates, score)
    return corrected_sentence

In [81]:
# Shuffle the misspelled dataset
random.shuffle(misspelled_dataset)

# Keep the first 10 sentences
subset_of_sentences = misspelled_dataset[:20]
subset_of_sentences = [sentence.strip() for sentence in subset_of_sentences]
# Filter out empty sentences (if needed)
subset_of_sentences = [sentence for sentence in subset_of_sentences if sentence]

# Now, 'subset_of_sentences' contains a shuffled subset of 10 sentences from the misspelled dataset.
subset_of_sentences

["'it isn't,' said the cayirpillar.",
 "'i'm getting tired og yhis. i vote the young lady tells us a syory.'",
 'qhole',
 "won't tamk about cats ot togs eitner, if you don't like then!' whin the",
 'wad a table, with a larje tish of tarys upon it: they lpokid do good,',
 'oroposal.',
 "tnat ell the jurors were wriying down 'stupid thingd!' om their slatis,",
 'executet fot having missid tneir turns, and she tid not liki the lool',
 'them treeding on her toed when they pasded too close, and waving tneir',
 "last came a little fievle, squeaking voice. ('thay's bill.' thought",
 'converdation dropped, and thi party sat siment gor e monute, while alice',
 'tabbit returning, spmendidly ttedsed, woth a oair of wnite kid gloves in',
 'the mallets live flamingoes, and the soldpirs had to double themselves',
 "'uou dom't know much,' siid tne duchess; 'and that's a fact.'",
 "caytiously: 'but i tpn't undetstand. where did they draw tne treasle",
 '\'hm! no eccounting for tastes! sing ner "turtle

In [82]:
corrected_dataset = [correct_spelling_with_beam_search(sentence, max_distance, beam_width, generate_candidates, score) for sentence in subset_of_sentences]
# Remove leading and trailing spaces from each sentence
corrected_dataset = [sentence.strip() for sentence in corrected_dataset]

# Filter out empty sentences (if needed)
corrected_dataset = [sentence for sentence in corrected_dataset if sentence]

corrected_dataset

["'it n't said the caterpillar",
 "'if getting time of this i one the you made tell as a sort",
 'while',
 "n't time about cat of to hatter if you n't like then when the",
 'had a table with a large wish of as upon it they looked do good',
 'oroposal.',
 'that well the curious were thing down said things of their said',
 'feet for have said their turned and she did not like the look',
 'them thing on her the when they asked too come and thing their',
 "last came a little little something voice 'that bill thought",
 'converdation replied and the day at moment for i minute while alice',
 'rabbit getting spmendidly herself with a air of white did over in',
 'the called like flamingoes, and the sort had to would herself',
 "'you do know much said the duchess 'and that a face",
 "caytiously: 'but i n't undetstand. were did they day the please",
 "'m no eccounting for hatter king her turtle spoke will you old",
 'as she said this the looked down at her hand and was replied to see',
 'who of 

In [83]:
# Create a list of lists for the table
table_data = []
for misspelled, corrected in zip(subset_of_sentences, corrected_dataset):
    table_data.append([misspelled, corrected])

# Display the table
table = tabulate(table_data, headers=["Misspelled Sentence", "Corrected Sentence"], tablefmt="grid")
print(table)


+--------------------------------------------------------------------------+---------------------------------------------------------------------+
| Misspelled Sentence                                                      | Corrected Sentence                                                  |
+==========================================================================+=====================================================================+
| 'it isn't,' said the cayirpillar.                                        | 'it n't said the caterpillar                                        |
+--------------------------------------------------------------------------+---------------------------------------------------------------------+
| 'i'm getting tired og yhis. i vote the young lady tells us a syory.'     | 'if getting time of this i one the you made tell as a sort          |
+--------------------------------------------------------------------------+------------------------------------------

# Metrics

##  Word Error Rate (WER)

WER is a metric used to evaluate the performance of systems that generate or correct sequences of words, such as automatic speech recognition systems or spelling correction systems. It measures the number of word-level errors made by a system when comparing its output to the reference or ground truth text.

### $ WER = \frac{S + I + D}{N} $

* $ S $ : the number of substitutions
* $ i $ : the number of insertions
* $ D $ : the number of deletions
* N: the total number of words in the reference

In [84]:
def calculate_wer(reference, corrected):
    # Calculate Word Error Rate (WER)
    reference_words = reference.split()
    corrected_words = corrected.split()

    S = Levenshtein(reference_words, corrected_words)
    I = max(0, len(corrected_words) - len(reference_words))
    D = max(0, len(reference_words) - len(corrected_words))

    N = max(len(reference_words), len(corrected_words))

    wer = (S + I + D) / N

    return wer


In [85]:
total_wer = 0
num_sentences = len(subset_of_sentences)

for reference, corrected in zip(subset_of_sentences, corrected_dataset):
    wer = calculate_wer(reference, corrected)
    total_wer += wer

average_wer = total_wer / num_sentences

print("Average WER:", average_wer)

Average WER: 0.5498529248529249


## Character Error Rate (CER)
CER is similar to WER but operates at the character level. It measures the accuracy of character-level transcriptions or corrections. CER quantifies the number of character-level errors made by a system when comparing its output to the reference text. Errors include substitutions, insertions, and deletions of individual characters.

### $ WER = \frac{S + I + D}{N} $

* $ S $ : the number of character substitutions
* $ i $ : the number of character insertions
* $ D $ : the number of character deletions
* N: the total number of characterσ in the reference

In [86]:
def calculate_cer(reference, corrected):
    S = Levenshtein(reference, corrected)
    I = max(0, len(corrected) - len(reference))
    D = max(0, len(reference) - len(corrected))

    N = max(len(reference), len(corrected))

    cer = (S + I + D) / N

    return cer


In [87]:
# Assuming you have corrected_dataset and subset_of_sentences

total_cer = 0
num_sentences = len(subset_of_sentences)

for reference, corrected in zip(subset_of_sentences, corrected_dataset):
    # Calculate CER
    cer = calculate_cer(reference, corrected)
    total_cer += cer

average_cer = total_cer / num_sentences

print("Average CER:", average_cer)


Average CER: 0.3103985165941203


In both WER and CER, a lower error rate indicates better accuracy

In [88]:
data = [
    ["Average WER", average_wer],
    ["Average CER", average_cer]
]

# Create the table
table = tabulate(data, headers=["Metrics" , "Value"], tablefmt="grid")

# Print the table
print(table)

+-------------+----------+
| Metrics     |    Value |
+=============+==========+
| Average WER | 0.549853 |
+-------------+----------+
| Average CER | 0.310399 |
+-------------+----------+
